In [1]:
# loading transactions and storing in a list of lists

def load_transactions(path):
    Transactions = []
    count = 1
    with open(path, 'r') as df:
        for line in df:
            l = list(line.strip().split(','))
            Transactions.append(l)
            count += 1
            if count >= 100:
                break
    return Transactions

Loading Data

In [2]:
path = 'Datasets/Goods_service_dataset.csv'
Transactions = load_transactions(path)

In [3]:
print(Transactions)

[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29'], ['30', '31', '32'], ['33', '34', '35'], ['36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46'], ['38', '39', '47', '48'], ['38', '39', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58'], ['32', '41', '59', '60', '61', '62'], ['3', '39', '48'], ['63', '64', '65', '66', '67', '68'], ['32', '69'], ['48', '70', '71', '72'], ['39', '73', '74', '75', '76', '77', '78', '79'], ['36', '38', '39', '41', '48', '79', '80', '81'], ['82', '83', '84'], ['41', '85', '86', '87', '88'], ['39', '48', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101'], ['36', '38', '39', '48', '89'], ['39', '41', '102', '103', '104', '105', '106', '107', '108'], ['38', '39', '41', '109', '110'], ['39', '111', '112', '113', '114', '115', '116', '117', '118'], ['119', '120', '121', '122', '123', '12

Initialising the itemsets

In [32]:
C = {}
L = {}
order = [str(i) for i in range(0, 525)]
itemsize = 1
InvalidSets = {itemsize: []}
C.update({itemsize: [ [f] for f in order]})
print(C)

{1: [['0'], ['1'], ['2'], ['3'], ['4'], ['5'], ['6'], ['7'], ['8'], ['9'], ['10'], ['11'], ['12'], ['13'], ['14'], ['15'], ['16'], ['17'], ['18'], ['19'], ['20'], ['21'], ['22'], ['23'], ['24'], ['25'], ['26'], ['27'], ['28'], ['29'], ['30'], ['31'], ['32'], ['33'], ['34'], ['35'], ['36'], ['37'], ['38'], ['39'], ['40'], ['41'], ['42'], ['43'], ['44'], ['45'], ['46'], ['47'], ['48'], ['49'], ['50'], ['51'], ['52'], ['53'], ['54'], ['55'], ['56'], ['57'], ['58'], ['59'], ['60'], ['61'], ['62'], ['63'], ['64'], ['65'], ['66'], ['67'], ['68'], ['69'], ['70'], ['71'], ['72'], ['73'], ['74'], ['75'], ['76'], ['77'], ['78'], ['79'], ['80'], ['81'], ['82'], ['83'], ['84'], ['85'], ['86'], ['87'], ['88'], ['89'], ['90'], ['91'], ['92'], ['93'], ['94'], ['95'], ['96'], ['97'], ['98'], ['99'], ['100'], ['101'], ['102'], ['103'], ['104'], ['105'], ['106'], ['107'], ['108'], ['109'], ['110'], ['111'], ['112'], ['113'], ['114'], ['115'], ['116'], ['117'], ['118'], ['119'], ['120'], ['121'], ['122']

Create L1

In [33]:
# counts support of the itemset
def countOccurences(itemset, Transactions):
    count = 0
    for i in range(len(Transactions)):
        if set(itemset).issubset(set(Transactions[i])):
            count += 1
    return count

In [34]:
def getFreqItemSet(itemsets, Transactions, minSupport, prevInvalidSubsets):

    sizeInvalid = len(prevInvalidSubsets)
    L = []
    itemSetSupport = []
    newInvalidSets = []
    
    for i in range(len(itemsets)):
        invalid = False
        # checks if any subset of the current itemset does not meet the minimum support threshold
        if sizeInvalid > 0:
            for itemS in prevInvalidSubsets[sizeInvalid]:
                if set(itemS).issubset(set(itemsets[i])):
                    invalid = True
                    break
        # counts support for the itemset
        if not invalid:
            countSup = countOccurences(itemsets[i], Transactions)
            if countSup/len(Transactions) >= minSupport:
                L.append(itemsets[i])
                itemSetSupport.append(countSup)
            else:
                newInvalidSets.append(itemsets[i])
    return L, itemSetSupport, newInvalidSets

In [35]:
def printTable(T, support):
    print("Itemset           | Frequency")
    for i in range(len(T)):
        print("{}   :   {}".format(T[i], support[i]))

In [36]:
supportL = {}
minSupport = 0.03
minConf = 0.7
f, supp, newInvalidSets = getFreqItemSet(C[itemsize], Transactions, minSupport, InvalidSets)
InvalidSets.update({itemsize: newInvalidSets})
L.update({itemsize: f})
supportL.update({itemsize: supp})

In [37]:
printTable(L[1], supportL[1])

Itemset           | Frequency
['11']   :   3
['32']   :   10
['36']   :   8
['37']   :   3
['38']   :   26
['39']   :   57
['41']   :   24
['48']   :   46
['60']   :   3
['65']   :   5
['79']   :   3
['89']   :   4
['105']   :   4
['110']   :   3
['147']   :   3
['152']   :   4
['161']   :   3
['170']   :   3
['179']   :   3
['186']   :   3
['225']   :   4
['237']   :   3
['242']   :   3
['258']   :   3
['340']   :   3


In [38]:
def joinItemSetsHelper(it1, it2, order):
    it1.sort(key = lambda x: order.index(x))
    it2.sort(key = lambda x: order.index(x))
    
    for i in range(len(it1) - 1):
        # ensures join is happening between two same itemsets
        if it1[i] != it2[i]:
            return []
    # self joining of itemsets
    if order.index(it1[-1]) < order.index(it2[-1]):
        return it1 + [it2[-1]]
        
    return []  

In [41]:
def joinItemSets(setItemSets):
    C = []
    for i in range(len(setItemSets)):
        for j in range(i+1, len(setItemSets)):
            it = joinItemSetsHelper(setItemSets[i], setItemSets[j], order)
            if len(it) > 0:
                C.append(it)
    return C

In [42]:
k = itemsize + 1
convergence = False
# updates C for every iteration
while not convergence:
    C.update({ k : joinItemSets(L[k-1])})
    print("Table (C{}:  \n)".format(k))
    printTable(C[k], [countOccurences(it, Transactions) for it in C[k]])
    f, supp, newInvalidSets = getFreqItemSet(C[k], Transactions, minSupport, InvalidSets)
    InvalidSets.update({k: newInvalidSets})
    L.update({k: f})
    supportL.update({k : supp})
    if(len(L[k]) == 0):
        convergence = True
    else:
        print("Table (L{}:  \n)".format(k))
        printTable(L[k], supportL[k])
    k += 1

Table (C2:  
)
Itemset           | Frequency
['11', '32']   :   0
['11', '36']   :   0
['11', '37']   :   0
['11', '38']   :   0
['11', '39']   :   2
['11', '41']   :   1
['11', '48']   :   2
['11', '60']   :   0
['11', '65']   :   1
['11', '79']   :   0
['11', '89']   :   1
['11', '105']   :   0
['11', '110']   :   0
['11', '147']   :   0
['11', '152']   :   0
['11', '161']   :   0
['11', '170']   :   0
['11', '179']   :   0
['11', '186']   :   0
['11', '225']   :   0
['11', '237']   :   0
['11', '242']   :   0
['11', '258']   :   0
['11', '340']   :   0
['32', '36']   :   0
['32', '37']   :   0
['32', '38']   :   3
['32', '39']   :   6
['32', '41']   :   4
['32', '48']   :   6
['32', '60']   :   1
['32', '65']   :   0
['32', '79']   :   0
['32', '89']   :   0
['32', '105']   :   0
['32', '110']   :   0
['32', '147']   :   0
['32', '152']   :   2
['32', '161']   :   0
['32', '170']   :   1
['32', '179']   :   1
['32', '186']   :   0
['32', '225']   :   0
['32', '237']   :   1
['32', '

Generating association rules

In [43]:
from itertools import combinations, chain

def powerset(s):
    return list(chain.from_iterable(combinations(s, r) for r in range(1, len(s) + 1)))

In [44]:
assoRules = ""
for i in range(len(L[4])):
    s = powerset(L[4][i])
    s.pop()
    for j in s:
        S = set(j)
        I = set(L[4][i])
        I_S = set(I-S)
        suppI = countOccurences(I, Transactions)
        conf = suppI/countOccurences(S, Transactions)
        if conf >= minConf and suppI >= minSupport:
            assoRules += "Frequency Itemset: {}\n".format(I)
            assoRules += "Rule: {} => {}\n".format(list(S), list(I_S))
            assoRules += "Confidence: {0:2.3f}\n".format(conf)
            assoRules += "Support:{0:2.3f}\n".format(suppI/len(Transactions))

In [45]:
print(assoRules)

Frequency Itemset: {'48', '38', '39', '32'}
Rule: ['38', '32'] => ['48', '39']
Confidence: 1.000
Support:0.030
Frequency Itemset: {'48', '38', '39', '32'}
Rule: ['38', '39', '32'] => ['48']
Confidence: 1.000
Support:0.030
Frequency Itemset: {'48', '38', '39', '32'}
Rule: ['48', '38', '32'] => ['39']
Confidence: 1.000
Support:0.030
Frequency Itemset: {'48', '41', '39', '32'}
Rule: ['41', '32'] => ['48', '39']
Confidence: 0.750
Support:0.030
Frequency Itemset: {'48', '41', '39', '32'}
Rule: ['41', '39', '32'] => ['48']
Confidence: 1.000
Support:0.030
Frequency Itemset: {'48', '41', '39', '32'}
Rule: ['48', '41', '32'] => ['39']
Confidence: 1.000
Support:0.030
Frequency Itemset: {'48', '38', '39', '170'}
Rule: ['170'] => ['48', '38', '39']
Confidence: 1.000
Support:0.030
Frequency Itemset: {'48', '38', '39', '170'}
Rule: ['170', '38'] => ['48', '39']
Confidence: 1.000
Support:0.030
Frequency Itemset: {'48', '38', '39', '170'}
Rule: ['170', '39'] => ['48', '38']
Confidence: 1.000
Support:0